# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_one = "Files/WeatherData.csv"
file_one_df = pd.read_csv(file_one, encoding="ISO-8859-1")
file_one_df

,City,temperature,humidity,cloudiness,wind speed,latitude,longitude
0,Iranshahr,17.00,59,61,2.10,27.20,60.68
1,Kibre Mengist,14.97,52,100,1.09,5.88,38.98
2,East London,21.75,72,100,4.80,-33.02,27.91
3,Port Alfred,17.22,97,100,0.89,-33.59,26.89
4,Koidu,24.85,67,99,0.30,8.42,-10.83
...,...,...,...,...,...,...,...
539,Phan Thiet,18.76,90,0,1.41,10.93,108.10
540,Sola,27.30,71,83,6.68,-13.88,167.55
541,Saint-Augustin,-6.98,91,95,3.55,51.23,-58.65
542,Plettenberg Bay,18.14,84,54,4.16,-34.05,23.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = file_one_df[["latitude", "longitude"]].astype(float)
humidity = file_one_df["humidity"]

In [5]:
# Create a worldwide Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
low_humidity_cities = file_one_df.loc[file_one_df["humidity"] <= 20]
low_humidity_cities

,City,temperature,humidity,cloudiness,wind speed,latitude,longitude
79,Ndele,22.16,14,54,3.51,8.41,20.65
87,Bilma,17.56,16,0,3.22,18.69,12.92
92,Wukari,26.73,16,45,2.89,7.85,9.78
194,Tshane,27.48,15,0,0.98,-24.02,21.87
279,Puerto Madryn,34.58,17,18,5.98,-42.77,-65.04
324,Saint-Louis,26.39,10,100,4.14,16.33,-15.00
343,Sibut,24.50,18,100,2.53,5.72,19.07
473,Juba,26.18,18,98,2.64,4.85,31.58
524,Tera,25.45,14,95,4.08,14.01,0.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = low_humidity_cities
hotel_df['Hotel Name'] = ""
hotel_df

C:\Users\codyd\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,temperature,humidity,cloudiness,wind speed,latitude,longitude,Hotel Name
79,Ndele,22.16,14,54,3.51,8.41,20.65,
87,Bilma,17.56,16,0,3.22,18.69,12.92,
92,Wukari,26.73,16,45,2.89,7.85,9.78,
194,Tshane,27.48,15,0,0.98,-24.02,21.87,
279,Puerto Madryn,34.58,17,18,5.98,-42.77,-65.04,
324,Saint-Louis,26.39,10,100,4.14,16.33,-15.00,
343,Sibut,24.50,18,100,2.53,5.72,19.07,
473,Juba,26.18,18,98,2.64,4.85,31.58,
524,Tera,25.45,14,95,4.08,14.01,0.75,


In [8]:
# params dictionary to update each iteration
params = {
    "key": g_key,
    "radius": 5000,
    #"rankby": "distance",
    "types": "lodging"
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]
    print(lat, lng)

    # change location each iteration while leaving original params in place
    params["location"] = f'{lat},{lng}'
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make the api request
    response = requests.get(base_url, params=params)
    hotel_search = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_search["results"][0]["name"]
    
    except:
        print("Missing field/result... skipping.")

8.41 20.65
Missing field/result... skipping.
18.69 12.92


C:\Users\codyd\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


7.85 9.78
-24.02 21.87
Missing field/result... skipping.
-42.77 -65.04
16.33 -15.0
Missing field/result... skipping.
5.72 19.07
Missing field/result... skipping.
4.85 31.58
14.01 0.75


In [9]:
hotel_df.head(10)

,City,temperature,humidity,cloudiness,wind speed,latitude,longitude,Hotel Name
79,Ndele,22.16,14,54,3.51,8.41,20.65,
87,Bilma,17.56,16,0,3.22,18.69,12.92,Expeditions Ténére voyages
92,Wukari,26.73,16,45,2.89,7.85,9.78,Elimtop
194,Tshane,27.48,15,0,0.98,-24.02,21.87,
279,Puerto Madryn,34.58,17,18,5.98,-42.77,-65.04,Hotel Tolosa
324,Saint-Louis,26.39,10,100,4.14,16.33,-15.00,
343,Sibut,24.50,18,100,2.53,5.72,19.07,
473,Juba,26.18,18,98,2.64,4.85,31.58,ROYAL PALACE HOTEL - JUBA
524,Tera,25.45,14,95,4.08,14.01,0.75,Hotel Daressalam Téra


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [11]:
# Add marker layer ontop of heat map


# Display Map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))